In [1]:
import os
import pandas as pd
import numpy as np

In [13]:
from datetime import date
today = date.today().strftime('%y%m%d')
print(today)

191119


In [96]:
WIKI_KO_DATA = './data/cc.ko.300.bin.gz'
KO_MODEL_PATH = f'./model/{today}_ko_model'
TRAINED_KO_MODEL = './model/191119_ko_model'
DATA_PATH = './data/10000_data.tsv'
MODEL_PATH = './model/10000_doc_model'

In [15]:
if not os.path.exists('./model'):
    os.makedirs('./model')

### Gensim FastText
- cc.ko.300.bin.gz : [Fast Text에서 제공하는 위키백과 한국어 데이터](https://fasttext.cc/docs/en/crawl-vectors.html)

In [5]:
if os.path.isfile(TRAINED_KO_MODEL):
    # 학습된 파일이 있는 경우
    print('trained model exist.')
    from gensim.models import FastText
    ko_model = FastText.load(TRAINED_KO_MODEL)
    print('model loadedß')
    
else:
    # 학습된 파일이 없는 경우
    print('start to train')
    from gensim import models
    if os.path.isfile(WIKI_KO_DATA):
        ko_model = models.fasttext.load_facebook_model(WIKI_KO_DATA)
        ko_model.save(KO_MODEL_PATH)
        print('model saved')
    else:
        print('no wiki_ko_data. please download cc.ko.300.bin.gz from [https://fasttext.cc/docs/en/crawl-vectors.html]')

trained model exist


In [8]:
for w, sim in ko_model.wv.similar_by_word('파이썬', 10):
    print(f'{w}: {sim}')

Python: 0.565061628818512
자이썬: 0.5624369382858276
레일스: 0.5598082542419434
파이썬을: 0.5595801472663879
언어용: 0.5288202166557312
파이썬의: 0.5250024795532227
프로그래밍: 0.5225088000297546
wxPython: 0.5222088098526001
파이썬이나: 0.5201171636581421
함수형: 0.5187377333641052


In [21]:
print(f'vocab size : {len(ko_model.wv.vocab)}')

vocab size : 2000000


In [22]:
data = pd.read_csv(DATA_PATH, sep='\t', usecols=['label', 'sentence'])
data.sample(5)

,label,sentence
3054,1,blockchain 블록체인 아이덴티티 자주적신원 blockchain and ide...
8650,1,java algorithm 새로 나온 자바 11 12 자바 gc 알고리즘 자바11e...
8386,1,android android jetpack constraintlayout const...
1199,1,golang pdf 문서 생성하는 gofpdf 라이브러리 알아보기 오늘은 golan...
8217,0,내가 겪은 미국에서 한류 강남스타일부터 bts 그리고 baby shark까지 미국에...


In [23]:
data.sentence

0       todo 리스트를 1달간 사용해본 후기 왜 todo 리스트를 사용했나요 위 질문에 ...
1       양평 블룸비스타 콩순이 패밀리룸이 궁금하신분에게 드리는 후기 늦은 여름 휴가로 양평...
2       javascript js 자바스크립트 문자열 메소드 javascript 문자열 메소...
3       우리 시장에는 어떤 교육이 필요할까요 1 시장적합형 교육과 기술적합형 교육 며칠전 ...
4       저 많은 국민은행 앱들은 대체 다 어디다 쓰는 걸까 대략 정리 구글플래이 스토어에서...
5       java java 8 람다 표현식과 주의 사항 안녕하세요 오늘부터는 java8에 관...
6       cache 16장 트랜잭션과 락 2차 캐시 현업에서 주문 결제 등등 민감한 데이터를...
7       swift lets swift 2019에 다녀왔습니다 1년에 한번 있는 ios개발자...
8       건설 현장 안전관리를 위한 iot 기반의 위험구역 경보 시스템 마침 지금 개발하는 ...
9       kstartup 그랜드챌린지ksgc 오픈하우스 행사와 참가기업 kstartup gr...
10      intellij 실행시 최근 종료한 프로젝트 바로 실행되지 않게 하기 intelli...
11      git 20191112 til with brumm 1 git repository 생...
12      golang timed rate limiter gorate 라이브러리 알아보기 오늘...
13      javascript 모던자바스크립트 25 심볼 타입 이 글은 번역 및 정리 글입니다...
14       마지막 7개월 초심 잃지 말자 방통대 졸업 정보처리기사 실기 msa인프라 포스팅 살빼기
15      cc 언어 및 코딩 관련 이야기 1 문법 함정 cc에서 연산자로 쓰이는 토큰문자들 ...
16      oracle oracle 오라클 insert문 사용법 예제 adsense1 disp...
17      김포공항김포

In [24]:
def sentenceToVector(sentence):
    words = sentence.split(' ')
    words_num = len(words)
    vector = np.zeros(300) # ko_model shape가 300
    for word in words:
        vector += ko_model[word]
    return vector/words_num
data['vector'] = data.sentence.apply(sentenceToVector)

/Users/apple/.virtualenvs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [26]:
def numberToOneHot(num):
    oneHot = [0]*2
    oneHot[num] = 1
    return oneHot

data['oneHot'] = data.label.apply(numberToOneHot)

In [27]:
data.head(3)

,label,sentence,vector,oneHot
0,1,todo 리스트를 1달간 사용해본 후기 왜 todo 리스트를 사용했나요 위 질문에 ...,"[0.013710441264420491, 0.0032995063650540692, ...","[0, 1]"
1,0,양평 블룸비스타 콩순이 패밀리룸이 궁금하신분에게 드리는 후기 늦은 여름 휴가로 양평...,"[0.00013062362416590355, -0.032540909874690126...","[1, 0]"
2,1,javascript js 자바스크립트 문자열 메소드 javascript 문자열 메소...,"[-0.04456514536979653, 0.00961365589007203, 0....","[0, 1]"


In [41]:
data.vector[0].shape

(300,)

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.vector, data.label, test_size=0.33, random_state=321)

In [91]:
X_train = pd.DataFrame(X_train)
X_train = np.array(X_train['vector'].tolist())
print(X_train.shape)
X_train = X_train.reshape(X_train.shape[0], 300, 1)
print(X_train.shape)

(6700, 300)
(6700, 300, 1)


In [92]:
X_test = pd.DataFrame(X_test)
X_test = np.array(X_test['vector'].tolist())
print(X_test.shape)
X_test = X_test.reshape(X_test.shape[0], 300, 1)
print(X_test.shape)

(3300, 300)
(3300, 300, 1)


In [35]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

In [95]:
from keras import backend as K 
K.clear_session()

model = Sequential()
# model.add(Embedding(len(ko_model.wv.vocab), 300)) # 임베딩 벡터의 차원은 300 <- fast text 쓰기 위해서 안씀
model.add(Dense(300, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(SimpleRNN(32)) # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=10, batch_size=60, validation_split=0.3)

Using TensorFlow backend.
W1119 11:57:32.391843 4538258880 deprecation_wrapper.py:119] From /Users/apple/.virtualenvs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1119 11:57:32.393152 4538258880 deprecation_wrapper.py:119] From /Users/apple/.virtualenvs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1119 11:57:32.395328 4538258880 deprecation_wrapper.py:119] From /Users/apple/.virtualenvs/ml/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Train on 4690 samples, validate on 2010 samples
Epoch 1/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.5756 - acc: 0.7232 - val_loss: 0.5453 - val_acc: 0.7264
Epoch 2/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.5052 - acc: 0.7546 - val_loss: 0.4876 - val_acc: 0.7682
Epoch 3/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.4590 - acc: 0.7855 - val_loss: 0.4634 - val_acc: 0.7731
Epoch 4/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.4377 - acc: 0.7964 - val_loss: 0.5314 - val_acc: 0.7468
Epoch 5/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.4351 - acc: 0.8002 - val_loss: 0.4939 - val_acc: 0.7647
Epoch 6/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.4332 - acc: 0.7987 - val_loss: 0.4706 - val_acc: 0.7746
Epoch 7/10
4690/4690 [==============================] - 8s 2ms/sample - loss: 0.4207 - acc: 0.8047 - val_loss: 0.4510 - val_acc: 0.7811


In [97]:
model.save(MODEL_PATH)

In [ ]:
model.predict('')